In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import keras
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn import metrics
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
START_DATE= '2015-01-01'
STOCK= 'AAPL'
def regression_eval(y_true,y_pred):
    variance_score= metrics.explained_variance_score(y_true, y_pred)
    mse= np.sqrt(metrics.mean_squared_error(y_true, y_pred))
    r2= metrics.r2_score(y_true, y_pred)

dff= yf.download(STOCK, 
                      start=START_DATE, 
                     
                      progress=False,)

df= dff[['Close','Volume']]
df['Date']= dff.index
df.reset_index(drop=True, inplace=True)
df


,Close,Volume,Date
0,27.594999,165613600,2014-12-31
1,27.332500,212818400,2015-01-02
2,26.562500,257142000,2015-01-05
3,26.565001,263188400,2015-01-06
4,26.937500,160423600,2015-01-07
...,...,...,...
1760,179.289993,79144300,2021-12-28
1761,179.380005,62348900,2021-12-29
1762,178.199997,59773000,2021-12-30
1763,177.570007,64025500,2021-12-31


In [3]:
training_data=df.iloc[:,:2]
from sklearn.preprocessing import MinMaxScaler
sc_c = MinMaxScaler(feature_range = (0, 1))

training_set_scaled= sc_c.fit_transform(training_data)





X = []
y = []
for i in range(1, len(df)):
    X.append(training_set_scaled [i-1:i, 0])
    y.append(training_set_scaled [i, 0])
X = np.asarray(X)
y = np.asarray(y)


split = int(0.7 * len(X))
X_train = X[:split]
y_train = y[:split]
X_test = X[split:]
y_test = y[split:]


X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


inputs = keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
x = keras.layers.LSTM(150, return_sequences= True)(inputs)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.LSTM(150, return_sequences=True)(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.LSTM(150)(x)
outputs = keras.layers.Dense(1, activation='linear')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss="mse")
model.summary()
history = model.fit(
    X_train, y_train,
    epochs = 20,
    batch_size = 32,
    validation_split = 0.2
)
predicted = model.predict(X)
test_predicted=[pred[0] for pred in predicted]
close=[close[0] for close in training_set_scaled]


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 1)]            0         
                                                                 
 lstm (LSTM)                 (None, 1, 150)            91200     
                                                                 
 dropout (Dropout)           (None, 1, 150)            0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 150)            180600    
                                                                 
 dropout_1 (Dropout)         (None, 1, 150)            0         
                                                                 
 lstm_2 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 1)                 151   

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit_transform(training_data[['Close']])

array([[0.03142544],
       [0.0297789 ],
       [0.02494904],
       ...,
       [0.97610163],
       [0.97214999],
       [1.        ]])

In [5]:
df_predicted= df[1:][['Date']]
df_predicted['prediction']=test_predicted
df_predicted['Close']=close[1:]

In [6]:
df_predicted

,Date,prediction,Close
1,2015-01-02,0.032073,0.029779
2,2015-01-05,0.030429,0.024949
3,2015-01-06,0.025609,0.024965
4,2015-01-07,0.025625,0.027301
5,2015-01-08,0.027957,0.033793
...,...,...,...
1760,2021-12-28,0.968659,0.982939
1761,2021-12-29,0.962709,0.983503
1762,2021-12-30,0.963225,0.976102
1763,2021-12-31,0.956464,0.972150


In [7]:
preds= np.asarray(df_predicted['prediction'].values)
preds= preds.reshape(-1, 1)
preds=scaler.inverse_transform(preds)

In [8]:
Close= np.asarray(df_predicted['Close'].values)
Close= Close.reshape(-1, 1)
Close=scaler.inverse_transform(Close)

In [9]:
def regression_eval(y_true,y_pred):
    variance_score= metrics.explained_variance_score(y_true, y_pred)
    mse= np.sqrt(metrics.mean_squared_error(y_true, y_pred))
    r2= metrics.r2_score(y_true, y_pred)
    return print(f'Varianace score (Best possible score is 1.0, lower values are worse.) : {variance_score}\nrmse : {mse}\nR2 (Best possible score is 1.0): {r2}')

regression_eval(Close,preds)

Varianace score (Best possible score is 1.0, lower values are worse.) : 0.9986496672770954
rmse : 1.490966374454728
R2 (Best possible score is 1.0): 0.9986393593672465


In [10]:
df_predicted= df[1:][['Date']]
df_predicted['prediction']=preds
df_predicted['Close']=Close


In [11]:
df_predicted

,Date,prediction,Close
1,2015-01-02,27.698183,27.332500
2,2015-01-05,27.436180,26.562500
3,2015-01-06,26.667791,26.565001
4,2015-01-07,26.670288,26.937500
5,2015-01-08,27.041981,27.972500
...,...,...,...
1760,2021-12-28,177.013382,179.289993
1761,2021-12-29,176.064911,179.380005
1762,2021-12-30,176.147064,178.199997
1763,2021-12-31,175.069290,177.570007


In [12]:
import plotly.express as px


def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()
interactive_plot(df_predicted, "Original Vs. Prediction")